In [30]:
from pypower.api import loadcase, runopf, ppoption
from pypower.idx_brch import *
from pypower.idx_bus import *
from pypower.idx_gen import *
from pypower.idx_gencost import *
import oct2py
import numpy as np
import os
from tqdm import tqdm
from oct2py import Oct2Py
octave = Oct2Py()
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
import pickle
import shutil
import cvxpy as cp
if shutil.which('latex') is None:
    LATEX_AVAILABLE = False
else:
    LATEX_AVAILABLE = True
if LATEX_AVAILABLE:
    plt.rcParams['font.size'] = 14
    plt.rcParams['text.usetex'] = True
    plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,amssymb}'
    
MAX_BUS = 1000 # upper limit of number of buses in cases to be considered

In [3]:
# get all cases in current directory
current_directory = os.getcwd()
all_files_and_directories = os.listdir(current_directory)
case_files = [os.path.join(current_directory, f) for f in all_files_and_directories if f.endswith('.m') and os.path.isfile(os.path.join(current_directory, f))]

cases, casenames = [], []
for cf in (t:=tqdm(case_files)):
    octave.source(os.path.basename(cf))
    cname = os.path.basename(cf).split('.')[0]
    num_buses = None
    for ci in cname.split('_'):
        if 'case' in ci:
            num_buses = int(''.join([chr for chr in ci.replace('case','',1) if chr.isdigit()]))
    if num_buses <= MAX_BUS:
        cases.append(loadcase(octave.feval(cname)))
        casenames.append(cname)
    t.set_description(f"Loaded {os.path.basename(cf)}")

Loaded pglib_opf_case793_goc.m: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66/66 [00:38<00:00,  1.73it/s]


In [32]:
this_case = cases[0]
to_str = lambda x: [str(int(xi)) for xi in x]
bus_list = to_str(this_case['bus'][:,BUS_I].tolist())
# branches
BR_ON = this_case['branch'][:,BR_STATUS].astype(bool).astype(int)
from_bus = to_str(this_case['branch'][:,F_BUS][BR_ON].tolist())
to_bus = to_str(this_case['branch'][:,T_BUS][BR_ON].tolist())
br_r = this_case['branch'][:,BR_R][BR_ON].tolist()
br_x = this_case['branch'][:,BR_X][BR_ON].tolist()
br_b = this_case['branch'][:,BR_B][BR_ON].tolist()
tap = this_case['branch'][:,TAP][BR_ON].tolist()
shift = this_case['branch'][:,TAP][BR_ON].tolist()
branch_list = [(f,t) for f,t in zip(from_bus,to_bus)]
# generators
GEN_ON = this_case['gen'][:,GEN_STATUS].astype(bool).astype(int)
gen_to_bus = this_case['gen'][:,GEN_BUS][GEN_ON]
gen_mbase = this_case['gen'][:,MBASE][GEN_ON]
gen_pmax, gen_pmin = this_case['gen'][:,PMAX][GEN_ON], this_case['gen'][:,PMIN][GEN_ON]
gen_qmax, gen_qmin = this_case['gen'][:,QMAX][GEN_ON], this_case['gen'][:,QMIN][GEN_ON]
# generator cost model
gmodel = this_case['gencost']



In [33]:
GEN_ON

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1])